<a href="https://colab.research.google.com/github/ruby199/build_DL_model/blob/main/Day1_02_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### CNN for Feature Extraction 
* image/video - 
e.g. pinterest(hashtag as labeling-supervise learning)
e.g. Adobe DeepFont (to detect specific designed font)

* paper review youtube channel:
https://www.youtube.com/@user-ow3gm9zd1b/featured


* CNN (mostly used in image classification)
* Fully-connected layer (어디 위치에 쓰이느냐에 따라 이름이 달라진다)

## Terminology
- **channel**: A channel is like a separate color or information in an image. In color images, there are different channels for red, green, and blue. Each channel holds specific color information.

- **Filter**: A filter is like a special tool used to modify an image. It can do things like making an image blurry, enhancing edges, or finding specific patterns. The filter uses a set of numbers to determine how it changes the image. 이미지에서 필요한/주요한 특징값만 추출하고 필요없는 특징값은 버리는 역할. CNN에서 feature값을 보고 판단/예측할 수 있다. filter안의 값들 = weight 값. *filter의 개수는 많을수록 좋고, 깊이도 깊을수록 더 좋다.* 중심부에서 더 많은 feature를 받게 된다.

- Stride: How far to go to the right/bottom to perform the next convolution (중앙값 뿐만 아니라 다른 곳에도 가중치를 두기 위해서, 즉 몇칸씩 이동할지)

- **Feature map**: A feature map is like a map that highlights specific features in an image. It's created by applying filters to the image. Each feature map shows the presence or intensity of a particular feature, such as edges or textures.

- **padding**: Padding is like adding extra space around an image. It helps to keep the image size consistent during certain operations. Padding can be done by adding extra pixels with specific values, like zeros, around the edges of the image.

- zero padding (0으로 테두리):  It involves adding extra rows and columns of zeros around the edges of an image or feature map before applying convolutional operations. 중앙필터 방지, & 가중으로 필터를 씌워야할 필요가 있을 때 - 이미지의 작아짐 방지. 

### CNN composed layers
- Convolutional Layer
- Fully Connected Layer
- Pooling Layer (cleaning up 정리정돈) - Max Pooling, Average Pooling. 짧은 시간안에 목표까지 도달할 수 있다.



## Text Classification
CNN: 영상 이미지 뿐만 아니라 모든 matrix data 가능, including text data
test -> 의미를 가지고 있는 vector값으로 표현...


### <추세>  
~2019 회귀 MLP. 이미지 CNN -> MLP MIXER , 자연어/시계열 RNN  -> CNN-> CNN + RNN

2021~
image, video - transformer (자연어에서 사용하던 transformer가 영상쪽에서 사용되자 기존 CNN 모두 이겨버림! 최고 성능을 내는 것은 결국 transformer이다)
BUT transformer (CNN보다 훨씬 더 많은 cost) - 1%~2% +성능이기 때문에 실무에서는 더 작은 model인 CNN 선호.

-> CNN + transformer (기반 model - 엄청난 성능, 그러나 엄청나게 크다) - 더 큰 transformer 모델이 무조건 성능이 좋기 때문에 성능적 개선을 연구하는 움직임은 침체. - 이제는 data적인 부분 (labeled/unlabeled data) 등 실무적인 부분에 대한 연구로 전향되고 있다. 

### MNIST dataset
- CNN사용 유무에 따른 연산량의 차이, 속도 차이 비교/연구

- Difference in computational complexity and speed comparison/research depending on the use of CNN.

- MNIST data: 다양한 case, data class의 정확성, data가 많고 다양하다. 실무에서 좋은 data 확보하는 것이 매우 중요하다!

In [32]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [33]:
mnist = tf.keras.datasets.mnist

In [34]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [35]:
# Normalize
X_train = X_train / 255.0
X_test = X_test / 255.0
# X_train, X_test norm(0-1)

### try training without CNN

In [36]:
model = tf.keras.models.Sequential()
#  multidimensional feature maps are reshaped into a single vector,(Here, it converts the 2D image into a 1D vector of size 784(28*28))
model.add(tf.keras.layers.Flatten(input_shape=(28, 28) )) # input shape: data하나의 크기가 관건이다. 
model.add(tf.keras.layers.Dense(128, activation='relu')) 
# input is 1D vector of size 784, each units in Dense layer has a weight for each input feature, 784*128=100,352 weights, 128 bias terms,
# Thus, 100,352+128=100,480 parameters

model.add(tf.keras.layers.Dropout(0.2)) # 특정 feature들을 가리고 학습하는 것. to prevent overfitting. 

model.add(tf.keras.layers.Dense(10 , activation ='softmax')) # This layer is defined with 10 units and uses the softmax activation function.
# input is 1D vector of size 128. Thus, 10*128 = 1,280 weights. & bias term of 10, 
# Thus, 1,280 + 10 = 1,290 parameters

model.summary() 
# Total: 100,480 + 1,290 = 101,770 parameters in the model

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (None, 784)               0         
                                                                 
 dense_12 (Dense)            (None, 128)               100480    
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Building loss function and optimizer

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [38]:
model.fit(X_train, y_train, epochs = 5, 
          validation_data=(X_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2936 - accuracy: 0.9139 - val_loss: 0.1359 - val_accuracy: 0.9579
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1446 - accuracy: 0.9571 - val_loss: 0.0984 - val_accuracy: 0.9701
Epoch 3/5
1875/1875 [==============================] - 10s 6ms/step - loss: 0.1075 - accuracy: 0.9680 - val_loss: 0.0860 - val_accuracy: 0.9737
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0881 - accuracy: 0.9732 - val_loss: 0.0822 - val_accuracy: 0.9754
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0764 - accuracy: 0.9753 - val_loss: 0.0804 - val_accuracy: 0.9752


- activation function: **linear to non-linear & normalization**
- 층이 깊어질수록 모델 학습이 안된다 - 해결 with activation function.
값의 범위를 줄이고, non-linear로 만드는 것. 

- hidden layer. output layer - 각각 어떤 function을 사용하는 것이 좋을까?

- [정리] hidden layer. output layer. (input layer는 이미 전 data preprocessing 작업 완료). output층에서는 결과값이 나와야하기 때문에 신호값으로 만들어주는 역할을 해야한다. 
- activation function을 사용하지 않을 때 = linear

### Activation functions

* sigmoid function - (0, 1) hidden & output layer 둘 다 사용 가능. 값의 범위가 무조건 0 또는 1로 만들어져야할때 사용. 두 가지일때는 softmax보다 sigmoid 쓰는 것이 좋다. 
* softmax - 모든 case에 대한 확률을 말해줘야한다. (e.g. 정상30 /위암40 /위염30) 세가지라면 softmax를 쓰는게 좋다. - output layer 확률값으로 바꿔주는 이유? 학습에 있어서 model이 얼마나 틀렸는지 알려주는 것이 중요하다!

* Tanh - (-1,1) sigmoid보다 값을 표현할 수 있는 범위가 넓어졌다.(공학적으로) 그러나 필요가 많지는 않다 (layer 1층을 더 쓰는게 편할 때가 많다) 더 다양한 시도등을 쓰기 좋을 때 사용 - e,g, 생성분야. 

* [Gradient vanishing]: activation function을 써도 layer층이 어느정도 깊어짐에 학습이 안되는 문제 직면 - 왜곡이 심해졌다. 원인이 activation function에 있었다. 

* Relu function(Rectified Linear Unit)  - layer가 깊어질수록 값에 대한 왜곡 문제 해결. 값의 범위를 줄인다 (음수 0으로 바꿔버림). 


### Loss function

* Cross Entropy - 0, 0, 1 - log1은 loss가 0이된다. 1에 가까워질수록 0에 가까워진다. 사실 맞췄는지 여부는 model에게 중요하지 않다. loss값이 중요. (정확도가 100%라고 loss가 0%는 아니다. 역은 통함) 분류일때는 cross entropy

* MSE - 수치계산할때는 MSE.

epoch - (hyperparameter) single pass through the entire training datasets전체 데이터 한번 돌면 1번
batch - subset of the training data used to update the weights of nn at each iteration within an epoch. update the weights after processing a batch of examples.(# of training examples included in each batch)

### Optimizer

stochastic gradient descent: (MINI-BATCHES)local minima에 빠지는 것도 방지(빠져나올 수 있는 확률을 만듬) .학습 속도가 빠르다.랜덤하게 batch를 선정해서 데이터를 가져옴.

momentum

adagrad(adaptive gradient): 방향성 고려. The learning rate is adapted component-wise to the parameters by incorporating knowledge of past observations.

adam (momentum + adagrad)

###  training with CNN

In [39]:
cnn_model = tf.keras.models.Sequential()
# input = (28,28)
cnn_model.add(tf.keras.layers.Conv2D(32, (3,3),padding = 'same',activation = 'relu',
                                     input_shape=(28,28,1)))
# hidden output shape= (28,28 ,32)
cnn_model.add(tf.keras.layers.MaxPooling2D((2,2), strides=1, padding='same'))
cnn_model.add(tf.keras.layers.Conv2D(32,(3,3),padding='same',activation='relu'))
cnn_model.add(tf.keras.layers.MaxPooling2D((2,2), strides=1, padding='same'))
cnn_model.add(tf.keras.layers.Conv2D(1,(3,3),padding='same',activation='relu'))

cnn_model.add(tf.keras.layers.MaxPooling2D((2,2), strides=1, padding='same'))

# hidden output shape= (28, 28 ,1) => feature , CNN X model = > pixel

cnn_model.add(tf.keras.layers.Flatten())

cnn_model.add(tf.keras.layers.Dense(128, activation='relu'))

cnn_model.add(tf.keras.layers.Dropout(0.2)) # model challenging 20 % hide

cnn_model.add(tf.keras.layers.Dense(10 , activation ='softmax'))

cnn_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 28, 28, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 28, 28, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 28, 28, 1)         289       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 28, 28, 1)       

In [43]:
cnn_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [44]:
expanded_X_train = tf.expand_dims(X_train , axis = -1 )
expanded_X_test = tf.expand_dims(X_test, axis=-1)

### model 준비 끝!

In [45]:
cnn_model.fit(expanded_X_train , y_train, epochs=5,

              validation_data=(expanded_X_test,y_test))

Epoch 1/5
1875/1875 [==============================] - 11s 5ms/step - loss: 0.2449 - accuracy: 0.9229 - val_loss: 0.0981 - val_accuracy: 0.9688
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1124 - accuracy: 0.9656 - val_loss: 0.0740 - val_accuracy: 0.9770
Epoch 3/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0867 - accuracy: 0.9731 - val_loss: 0.0555 - val_accuracy: 0.9829
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0760 - accuracy: 0.9755 - val_loss: 0.0606 - val_accuracy: 0.9818
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0643 - accuracy: 0.9796 - val_loss: 0.0577 - val_accuracy: 0.9812
